In [10]:
import random
import numpy as np
import csv

In [11]:
def start():
    AA = list(range(0,15,1));
    for i in range(iterateDifferentStart) : {
        createBasicInfo()
    }

In [12]:
def createBasicInfo():
    pricesCompanies = [round(random.uniform(6,16),2) for _ in range(numberPlayers)]
    qualityCompanies = [round(random.uniform(1,5)) for _ in range(numberPlayers)]
    ratingCompanies = [round(random.uniform(90,100),1) for _ in range(numberPlayers)]
    
    priceRanks = [1 +  
              sum([1 for i in range(numberPlayers) if pricesCompanies[j]> pricesCompanies[i] and i!=j]) +
              sum([1 for i in range(numberPlayers) if pricesCompanies[j]== pricesCompanies[i] and i!=j])/2
              for j in range(numberPlayers)]

    qualityRanks = [1 +  
              sum([1 for i in range(numberPlayers) if qualityCompanies[j]> qualityCompanies[i] and i!=j]) +
              sum([1 for i in range(numberPlayers) if qualityCompanies[j]== qualityCompanies[i] and i!=j])/2
              for j in range(numberPlayers)]

    ratingRanks = [1 +  
              sum([1 for i in range(numberPlayers) if ratingCompanies[j]> ratingCompanies[i] and i!=j]) +
              sum([1 for i in range(numberPlayers) if ratingCompanies[j]== ratingCompanies[i] and i!=j])/2
              for j in range(numberPlayers)]

    accumulatedProfit = [0 for _ in range(numberPlayers)]; # for example: sum([x+5 for x in range(numberPlayersForIndex)])
    createSimulationParameters(pricesCompanies, qualityCompanies, ratingCompanies, priceRanks, qualityRanks, ratingRanks, accumulatedProfit)

In [13]:
def calculateTimeOfAdjustmentsCompany(numberReactionEvent, company, timeBetweenMarketEvents):
    if numberReactionEvent==0:
        return(random.uniform(0,1)) 
    else: return(calculateTimeOfAdjustmentsCompany(numberReactionEvent-1, company, timeBetweenMarketEvents)+timeBetweenMarketEvents[company][numberReactionEvent]);
    
def calculateTimeOfCustomerArrival(numberCustomerEvent, timeBetweenCustomerEvents):
    if numberCustomerEvent==0:
        return(0)
    else:
        return(calculateTimeOfCustomerArrival(numberCustomerEvent-1, timeBetweenCustomerEvents) + timeBetweenCustomerEvents[numberCustomerEvent]);



In [14]:
def createSimulationParameters(pricesCompanies, qualityCompanies, ratingCompanies, priceRanks, qualityRanks, ratingRanks, accumulatedProfit):
    timeBetweenMarketEvents = [[random.uniform(0.8,1.2) if numberReactionEvents!= 1 else 1.1 for _ in range(numberReactionEvents)] for _ in range(numberPlayers)]


    timeOfAdjustmentsCompany = np.array([[calculateTimeOfAdjustmentsCompany(i,j, timeBetweenMarketEvents) for i in range(numberReactionEvents)] for j in range(numberPlayers)])

    priceAdjustmentsBeforeHorizon = set([timeOfAdjustmentsCompany.flatten()[i] for i in range(len(timeOfAdjustmentsCompany.flatten())) if timeOfAdjustmentsCompany.flatten()[i] < timeHorizon])
    priceAdjustmentsBeforeHorizon.add(timeHorizon)
    priceAdjustmentsBeforeHorizon = sorted(priceAdjustmentsBeforeHorizon)

    numberPriceAdjustmentsBeforeHorizon = len(priceAdjustmentsBeforeHorizon)

    timePriceAdjustments = [priceAdjustmentsBeforeHorizon[i] for i in range(numberPriceAdjustmentsBeforeHorizon)]

    #we have to be careful because of the first timeslot
    timeBetweenAdjustments = [timePriceAdjustments[i]-timePriceAdjustments[i-1] if i> 0 else timePriceAdjustments[i] for i in range(numberPriceAdjustmentsBeforeHorizon)]

    timeBetweenCustomerEvents = [meanDifferenceCustomerArrival*np.random.exponential() for _ in range(numberCustomerEvents)]

    timeOfCustomerArrival = [calculateTimeOfCustomerArrival(i, timeBetweenCustomerEvents) for i in range(numberCustomerEvents)]    
    
    companyAdjustingAtTime = [min(np.where(timeOfAdjustmentsCompany==timePriceAdjustments[i])[0])if timePriceAdjustments[i]!=timeHorizon else -1  for i in range(numberPriceAdjustmentsBeforeHorizon)]    
    
    randomCustomerScoringOfCompany = [[0 for _ in range(numberCustomerEvents)] for _ in range(numberPlayers)]

    randomCustomerScoringQuality = [random.uniform(0,1) for _ in range(numberCustomerEvents)]

    randomCustomerScoringRating = [random.uniform(0,0.5) for _ in range(numberCustomerEvents)]

    randomCustomerDecision = [None for _ in range(numberCustomerEvents)]

    
    for j in range(iterateSameStart): {
        createWatchParameters(pricesCompanies, qualityCompanies, ratingCompanies, priceRanks, qualityRanks, ratingRanks, accumulatedProfit,
                     timeBetweenMarketEvents, timeOfAdjustmentsCompany, priceAdjustmentsBeforeHorizon, numberPriceAdjustmentsBeforeHorizon,
                     timePriceAdjustments, timeBetweenAdjustments, timeBetweenCustomerEvents, timeOfCustomerArrival, companyAdjustingAtTime,
                     randomCustomerScoringOfCompany, randomCustomerScoringQuality, randomCustomerScoringRating, randomCustomerDecision)
    }

In [15]:
def createWatchParameters(pricesCompanies, qualityCompanies, ratingCompanies, priceRanks, qualityRanks, ratingRanks, accumulatedProfit,
                     timeBetweenMarketEvents, timeOfAdjustmentsCompany, priceAdjustmentsBeforeHorizon, numberPriceAdjustmentsBeforeHorizon,
                     timePriceAdjustments, timeBetweenAdjustments, timeBetweenCustomerEvents, timeOfCustomerArrival, companyAdjustingAtTime,
                     randomCustomerScoringOfCompany, randomCustomerScoringQuality, randomCustomerScoringRating, randomCustomerDecision):
    currentTimeProcess = 0

    countAdjustmentsCompany = 0

    watchedCompany = companyToWatch
    
    countAdjustmentsWatchedCompany = sum([1 for i in range(numberPriceAdjustmentsBeforeHorizon) if companyAdjustingAtTime[i] == watchedCompany])

    timeAdjustmentsWatchedCompany = [0 for _ in range(countAdjustmentsWatchedCompany+1)]

    timeBetweenAdjustmentsWatchedCompany = [timeAdjustmentsWatchedCompany[i] - timeAdjustmentsWatchedCompany[i-1] for i in range(1, countAdjustmentsWatchedCompany)]

    salesBetweenAdjustments = [0 for _ in range(countAdjustmentsWatchedCompany)]

    price = [pricesCompanies[watchedCompany] for _ in range(1,countAdjustmentsWatchedCompany+2)]

    quality = [qualityCompanies[watchedCompany] for _ in range(1,countAdjustmentsWatchedCompany+2)]

    rating = [ratingCompanies[watchedCompany] for _ in range(1,countAdjustmentsWatchedCompany+2)]

    pricesCompetitors = [[pricesCompanies[c] for _ in range(countAdjustmentsWatchedCompany+1)] for c in range(numberPlayers) ]

    qualityCompetitors = [[qualityCompanies[c] for _ in range(countAdjustmentsWatchedCompany+1)] for c in range(numberPlayers) ]

    ratingCompetitors = [[ratingCompanies[c] for _ in range(countAdjustmentsWatchedCompany+1)] for c in range(numberPlayers) ]
    
    runSimulation(pricesCompanies, qualityCompanies, ratingCompanies, priceRanks, qualityRanks, ratingRanks, accumulatedProfit,
                     timeBetweenMarketEvents, timeOfAdjustmentsCompany, priceAdjustmentsBeforeHorizon, numberPriceAdjustmentsBeforeHorizon,
                     timePriceAdjustments, timeBetweenAdjustments, timeBetweenCustomerEvents, timeOfCustomerArrival, companyAdjustingAtTime,
                     randomCustomerScoringOfCompany, randomCustomerScoringQuality, randomCustomerScoringRating, randomCustomerDecision,
                 currentTimeProcess, countAdjustmentsCompany, watchedCompany, countAdjustmentsWatchedCompany, timeAdjustmentsWatchedCompany,
                 timeBetweenAdjustmentsWatchedCompany, salesBetweenAdjustments, price, quality, rating, pricesCompetitors, qualityCompetitors,
                 ratingCompetitors)

In [50]:
def runSimulation(pricesCompanies, qualityCompanies, ratingCompanies, priceRanks, qualityRanks, ratingRanks, accumulatedProfit,
                     timeBetweenMarketEvents, timeOfAdjustmentsCompany, priceAdjustmentsBeforeHorizon, numberPriceAdjustmentsBeforeHorizon,
                     timePriceAdjustments, timeBetweenAdjustments, timeBetweenCustomerEvents, timeOfCustomerArrival, companyAdjustingAtTime,
                     randomCustomerScoringOfCompany, randomCustomerScoringQuality, randomCustomerScoringRating, randomCustomerDecision,
                 currentTimeProcess, countAdjustmentsCompany, watchedCompany, countAdjustmentsWatchedCompany, timeAdjustmentsWatchedCompany,
                 timeBetweenAdjustmentsWatchedCompany, salesBetweenAdjustments, price, quality, rating, pricesCompetitors, qualityCompetitors,
                 ratingCompetitors):
    arrival_out =[[0 for _ in range(numberCustomerEvents)] for _ in range(numberPriceAdjustmentsBeforeHorizon-1)]
    buying_out = [[0 for _ in range(numberCustomerEvents)] for _ in range(numberPriceAdjustmentsBeforeHorizon-1)]
    trajekt1_out = [[0 for _ in range(numberPlayers)] for _ in range(numberPriceAdjustmentsBeforeHorizon-1)]
    for i in range(1,numberPriceAdjustmentsBeforeHorizon): 
        for customer in [customer for customer in range(numberCustomerEvents) if timePriceAdjustments[i-1]< timeOfCustomerArrival[customer] < timePriceAdjustments[i]]:
            arrival_out[i-1][customer]=[timeOfCustomerArrival[customer],0,customer]
            for company in range(numberPlayers):
                randomCustomerScoringOfCompany[company][customer]= (pricesCompanies[company]+ 
                randomCustomerScoringQuality[customer] * qualityCompanies[company] +
                randomCustomerScoringRating[customer] * (100-ratingCompanies[company]));
            if min([randomCustomerScoringOfCompany[company][customer] for company in range(numberPlayers)]) < random.uniform(5,15):
                randomCustomerDecision[customer] = np.argmin([randomCustomerScoringOfCompany[company][customer] for company in range(numberPlayers)])
                accumulatedProfit[randomCustomerDecision[customer]] = accumulatedProfit[randomCustomerDecision[customer]] + pricesCompanies[randomCustomerDecision[customer]];
                buying_out[i-1][customer]=[timeOfCustomerArrival[customer], pricesCompanies[randomCustomerDecision[customer]], customer, randomCustomerDecision[customer]]
        currentTimeProcess= priceAdjustmentsBeforeHorizon[i]
        for company in range(numberPlayers):
            trajekt1_out[i-1][company] = [pricesCompanies[company], currentTimeProcess, company]
        if currentTimeProcess < timeHorizon:
            pricesCompanies[companyAdjustingAtTime[i]] = priceAdjustmentRules(companyAdjustingAtTime[i], pricesCompanies)
            priceRanks = [1 +  
                  sum([1 for i in range(numberPlayers) if pricesCompanies[j]> pricesCompanies[i] and i!=j]) +
                  sum([1 for i in range(numberPlayers) if pricesCompanies[j]== pricesCompanies[i] and i!=j])/2
                  for j in range(numberPlayers)]
        '''if random.uniform(0,1)<0.2:
            qualityCompanies[companyAdjustingAtTime[i]] = round(random.uniform(0.5,5.5))
            ratingCompanies[companyAdjustingAtTime[i]] = round(random.uniform(90,100),1)
            qualityRanks = [1 +  
                  sum([1 for i in range(numberPlayers) if qualityCompanies[j]> qualityCompanies[i] and i!=j]) +
                  sum([1 for i in range(numberPlayers) if qualityCompanies[j]== qualityCompanies[i] and i!=j])/2
                  for j in range(numberPlayers)]

            ratingRanks = [1 +  
                  sum([1 for i in range(numberPlayers) if ratingCompanies[j]> ratingCompanies[i] and i!=j]) +
                  sum([1 for i in range(numberPlayers) if ratingCompanies[j]== ratingCompanies[i] and i!=j])/2
                  for j in range(numberPlayers)]'''
        if companyAdjustingAtTime[i]==watchedCompany:
            countAdjustmentsCompany = countAdjustmentsCompany+1
            timeAdjustmentsWatchedCompany[countAdjustmentsCompany]=currentTimeProcess
            price[countAdjustmentsCompany] = pricesCompanies[watchedCompany]
            quality[countAdjustmentsCompany] = qualityCompanies[watchedCompany]
            rating[countAdjustmentsCompany] = ratingCompanies[watchedCompany]
            salesBetweenAdjustments[countAdjustmentsCompany-1]= sum([1 for c in range(numberCustomerEvents) if timeAdjustmentsWatchedCompany[countAdjustmentsCompany-1]<timeOfCustomerArrival[c]<timeAdjustmentsWatchedCompany[countAdjustmentsCompany] and watchedCompany==randomCustomerDecision[c]])
            for c in range(numberPlayers):
                if c!= watchedCompany:
                    pricesCompetitors[c][countAdjustmentsCompany]= pricesCompanies[c]
                    qualityCompetitors[c][countAdjustmentsCompany]= qualityCompanies[c]
                    ratingCompetitors[c][countAdjustmentsCompany]= ratingCompanies[c];
    #print(salesBetweenAdjustments)
    
    with open('arrival_out_S3.txt', 'w') as file:
        writer = csv.writer(file, delimiter='\t')
        for w in range(numberPriceAdjustmentsBeforeHorizon-1):
            currentSim = list(filter(lambda x : x != 0, arrival_out[w]))
            for c in range(len(currentSim)):
                line = list(currentSim[c])
                writer.writerow(line)
            
    with open('buying_out_S3.txt', 'w') as file:
        writer = csv.writer(file, delimiter='\t')
        for w in range(numberPriceAdjustmentsBeforeHorizon-1):
            currentSim = list(filter(lambda x : x != 0, buying_out[w]))
            for c in range(len(currentSim)):
                line = list(currentSim[c])
                writer.writerow(line)
            
    with open('trajekt1_out_S3.txt', 'w') as file:
        writer = csv.writer(file, delimiter='\t')
        for w in range(numberPriceAdjustmentsBeforeHorizon-1):
            currentSim = list(filter(lambda x : x != 0, trajekt1_out[w]))
            for k in range(len(currentSim)):
                line= list(currentSim[k])
                writer.writerow(line)

    

In [51]:
def priceAdjustmentRules(c, pricesCompanies):
    return {
        '0' : 10 if pricesCompanies[c]<5 else min([max(4.9, pricesCompanies[company]-0.5) for company in range(numberPlayers) if company != c]),
        '1' : 9 if pricesCompanies[c]<4 else min([max(3.9, pricesCompanies[company]-0.5) for company in range(numberPlayers) if company != c]),
        '2' : 12 if pricesCompanies[c]<6 else min([max(5.9, pricesCompanies[company]-0.5) for company in range(numberPlayers) if company != c]),
        '3' : round(random.uniform(11,11),2),
        '4' : round(random.uniform(13,13),2)
    }[str(c)]

numberPlayers=5
timeHorizon=1
numberReactionEvents=100
meanDifferenceCustomerArrival=0.05
numberCustomerEvents=300
iterateDifferentStart=100
iterateSameStart=100
companyToWatch=0
start()